In [1]:
import os
import glob
import json 
import random
from pathlib import Path
from difflib import SequenceMatcher


import cv2
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from matplotlib import pyplot, patches

In [2]:
def read_bbox_and_words(Pathh):
  bbox_and_words_list = []

  with open(Pathh, 'r', errors='ignore') as f:
    for line in f.read().splitlines():
      if len(line) == 0:
        continue
        
      split_lines = line.split(",")

      bbox = np.array(split_lines[0:8], dtype=np.int32)
      text = ",".join(split_lines[8:])

      # From the splited line we save (filename, [bounding box points], text line).
      # The filename will be useful in the future
      bbox_and_words_list.append([Pathh, *bbox, text])
    
  dataframe = pd.DataFrame(bbox_and_words_list, columns=['filename', 'x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'line'], dtype=np.int16)
  dataframe = dataframe.drop(columns=['x1', 'y1', 'x3', 'y3'])

  return dataframe
print("== File content ==")
!head -n 5 "{bbox_file_path}"
bbox = read_bbox_and_words(Pathh='SROIE2019/test/box/X51005268408.txt')
print("\n== Dataframe ==")
bbox.head(5)

== File content ==
head: {bbox_file_path}: No such file or directory

== Dataframe ==


/var/folders/jg/pktw8l5s32xgj2428mt_gdxr0000gn/T/ipykernel_13679/2817273887.py:18: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  dataframe = pd.DataFrame(bbox_and_words_list, columns=['filename', 'x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'line'], dtype=np.int16)


,filename,x0,y0,x2,y2,line
0,SROIE2019/test/box/X51005268408.txt,139,282,539,315,99 SPEED MART S/B (S19537-X)
1,SROIE2019/test/box/X51005268408.txt,156,319,541,357,"LOT P.T. 2811, JALAN ANGSA,"
2,SROIE2019/test/box/X51005268408.txt,243,359,443,390,TAMAN BERKELEY
3,SROIE2019/test/box/X51005268408.txt,201,397,505,435,"41150 KLANG, SELANGOR"
4,SROIE2019/test/box/X51005268408.txt,232,436,477,470,1413-SETIA ALAM 2


In [3]:
def read_entities(path: Path):
  with open(path, 'r') as f:
    data = json.load(f)

  dataframe = pd.DataFrame([data])
  return dataframe


# Example usage
entities_file_path = 'SROIE2019/test/entities/X51005200931.txt'
print("== File content ==")
!head "{entities_file_path}"

entities = read_entities(path=entities_file_path)
print("\n\n== Dataframe ==")
entities

== File content ==
{
    "company": "PERNIAGAAN ZHENG HUI",
    "date": "09/02/2018",
    "address": "NO.59 JALAN PERMAS 9/5 BANDAR BARU PERMAS JAYA 81750 JOHOR BAHRU",
    "total": "436.20"
}

== Dataframe ==


,company,date,address,total
0,PERNIAGAAN ZHENG HUI,09/02/2018,NO.59 JALAN PERMAS 9/5 BANDAR BARU PERMAS JAYA...,436.20


In [4]:
# Assign a label to the line by checking the similarity
# of the line and all the entities
def assign_line_label(line: str, entities: pd.DataFrame):
    line_set = line.replace(",", "").strip().split()
    for i, column in enumerate(entities):
        entity_values = entities.iloc[0, i].replace(",", "").strip()
        entity_set = entity_values.split()
        
        
        matches_count = 0
        for l in line_set:
            if any(SequenceMatcher(a=l, b=b).ratio() > 0.8 for b in entity_set):
                matches_count += 1
            
            if (column.upper() == 'ADDRESS' and (matches_count / len(line_set)) >= 0.5) or \
               (column.upper() != 'ADDRESS' and (matches_count == len(line_set))) or \
               matches_count == len(entity_set):
                return column.upper()

    return "O"


line = bbox.loc[1,"line"]
label = assign_line_label(line, entities)
print("Line:", line)
print("Assigned label:", label)

Line: LOT P.T. 2811, JALAN ANGSA,
Assigned label: O


In [5]:
def assign_labels(words: pd.DataFrame, entities: pd.DataFrame):
    max_area = {"TOTAL": (0, -1), "DATE": (0, -1)}  # Value, index
    already_labeled = {"TOTAL": False,
                       "DATE": False,
                       "ADDRESS": False,
                       "COMPANY": False,
                       "O": False
    }

    # Go through every line in $words and assign it a label
    labels = []
    for i, line in enumerate(words['line']):
        label = assign_line_label(line, entities)

        already_labeled[label] = True
        if (label == "ADDRESS" and already_labeled["TOTAL"]) or \
           (label == "COMPANY" and (already_labeled["DATE"] or already_labeled["TOTAL"])):
            label = "O"

        # Assign to the largest bounding box
        if label in ["TOTAL", "DATE"]:
            x0_loc = words.columns.get_loc("x0")
            bbox = words.iloc[i, x0_loc:x0_loc+4].to_list()
            area = (bbox[2] - bbox[0]) + (bbox[3] - bbox[1])

            if max_area[label][0] < area:
                max_area[label] = (area, i)

            label = "O"

        labels.append(label)

    labels[max_area["DATE"][1]] = "DATE"
    labels[max_area["TOTAL"][1]] = "TOTAL"

    words["label"] = labels
    return words


# Example usage
bbox_labeled = assign_labels(bbox, entities)
bbox_labeled.head(20)

,filename,x0,y0,x2,y2,line,label
0,SROIE2019/test/box/X51005268408.txt,139,282,539,315,99 SPEED MART S/B (S19537-X),O
1,SROIE2019/test/box/X51005268408.txt,156,319,541,357,"LOT P.T. 2811, JALAN ANGSA,",O
2,SROIE2019/test/box/X51005268408.txt,243,359,443,390,TAMAN BERKELEY,O
3,SROIE2019/test/box/X51005268408.txt,201,397,505,435,"41150 KLANG, SELANGOR",O
4,SROIE2019/test/box/X51005268408.txt,232,436,477,470,1413-SETIA ALAM 2,O
5,SROIE2019/test/box/X51005268408.txt,176,474,537,509,GST ID. NO : 000181747712,O
6,SROIE2019/test/box/X51005268408.txt,165,550,562,590,INVOICE NO : 18222/102/T0341,O
7,SROIE2019/test/box/X51005268408.txt,81,641,188,671,03:29PM,O
8,SROIE2019/test/box/X51005268408.txt,326,636,416,665,562936,O
9,SROIE2019/test/box/X51005268408.txt,542,622,655,655,20-11-17,O


In [6]:
import pandas as pd

def split_line(line: pd.Series):
    line_copy = line.copy()

    line_str = line_copy.loc["line"]
    words = line_str.split(" ")

    # Filter unwanted tokens
    words = [word for word in words if len(word) >= 1]

    x0, y0, x2, y2 = line_copy.loc[['x0', 'y0', 'x2', 'y2']]
    bbox_width = x2 - x0

    new_lines = []
    for word in words:
        word_width = int(bbox_width * len(word) / len(line_str))
        x2_new = x0 + word_width

        new_line = line_copy.copy()
        new_line.at['x0'] = x0
        new_line.at['x2'] = x2_new
        new_line.at['line'] = word
        new_lines.append(new_line.to_list())

        x0 = x2_new + 5

    return new_lines


# Example usage
new_lines = split_line(bbox_labeled.loc[1])
print("Original row:")
display(bbox_labeled.loc[1:1,:])

print("Splitted row:")
pd.DataFrame(new_lines, columns=bbox_labeled.columns)

Original row:


,filename,x0,y0,x2,y2,line,label
1,SROIE2019/test/box/X51005268408.txt,156,319,541,357,"LOT P.T. 2811, JALAN ANGSA,",O


Splitted row:


,filename,x0,y0,x2,y2,line,label
0,SROIE2019/test/box/X51005268408.txt,156,319,198,357,LOT,O
1,SROIE2019/test/box/X51005268408.txt,203,319,260,357,P.T.,O
2,SROIE2019/test/box/X51005268408.txt,265,319,336,357,"2811,",O
3,SROIE2019/test/box/X51005268408.txt,341,319,412,357,JALAN,O
4,SROIE2019/test/box/X51005268408.txt,417,319,502,357,"ANGSA,",O


In [11]:
from time import perf_counter
def dataset_creator(folder: Path):
  bbox_folder = folder / 'box'
  entities_folder = folder / 'entities'
  img_folder = folder / 'img'

  # Sort by filename so that when zipping them together
  # we don't get some other file (just in case)
  entities_files = sorted(entities_folder.glob("*.txt"))
  bbox_files = sorted(bbox_folder.glob("*.txt"))
  img_files = sorted(img_folder.glob("*.jpg"))

  data = []

  print("Reading dataset:")
  for bbox_file, entities_file, img_file in tqdm(zip(bbox_files, entities_files, img_files), total=len(bbox_files)):            
    # Read the files
    bbox = read_bbox_and_words(bbox_file)
    entities = read_entities(entities_file)
    image = Image.open(img_file)

    # Assign labels to lines in bbox using entities
    bbox_labeled = assign_labels(bbox, entities)
    del bbox

    # Split lines into separate tokens
    new_bbox_l = []
    for index, row in bbox_labeled.iterrows():
      new_bbox_l += split_line(row)
    new_bbox = pd.DataFrame(new_bbox_l, columns=bbox_labeled.columns, dtype=np.int16)
    del bbox_labeled


    # Do another label assignment to keep the labeling more precise 
    for index, row in new_bbox.iterrows():
      label = row['label']

      if label != "O":
        entity_values = entities.iloc[0, entities.columns.get_loc(label.lower())]
        entity_set = entity_values.split()
        
        if any(SequenceMatcher(a=row['line'], b=b).ratio() > 0.7 for b in entity_set):
            label = "S-" + label
        else:
            label = "O"
      
      new_bbox.at[index, 'label'] = label

    width, height = image.size
  
    data.append([new_bbox, width, height])

  return data

In [12]:
dataset_train = dataset_creator(Path("SROIE2019/train"))
dataset_test = dataset_creator(Path("SROIE2019/test"))

Reading dataset:


  0%|          | 0/626 [00:00<?, ?it/s]/var/folders/jg/pktw8l5s32xgj2428mt_gdxr0000gn/T/ipykernel_13679/2817273887.py:18: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  dataframe = pd.DataFrame(bbox_and_words_list, columns=['filename', 'x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'line'], dtype=np.int16)
/var/folders/jg/pktw8l5s32xgj2428mt_gdxr0000gn/T/ipykernel_13679/3543968369.py:30: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  new_bbox = pd.DataFrame(new_bbox_l, columns=bbox_labeled.columns, dtype=np.int16)
/var/folders/jg/pktw8l5s32xgj2428mt_gdxr0000gn/T/ipykernel_13679/2817273887.py:18: FutureWarning: Could not

Reading dataset:


  0%|          | 0/347 [00:00<?, ?it/s]/var/folders/jg/pktw8l5s32xgj2428mt_gdxr0000gn/T/ipykernel_13679/2817273887.py:18: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  dataframe = pd.DataFrame(bbox_and_words_list, columns=['filename', 'x0', 'y0', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'line'], dtype=np.int16)
/var/folders/jg/pktw8l5s32xgj2428mt_gdxr0000gn/T/ipykernel_13679/3543968369.py:30: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  new_bbox = pd.DataFrame(new_bbox_l, columns=bbox_labeled.columns, dtype=np.int16)
/var/folders/jg/pktw8l5s32xgj2428mt_gdxr0000gn/T/ipykernel_13679/2817273887.py:18: FutureWarning: Could not

In [13]:
def normalize(points: list, width: int, height: int) -> list:
  x0, y0, x2, y2 = [int(p) for p in points]
  
  x0 = int(1000 * (x0 / width))
  x2 = int(1000 * (x2 / width))
  y0 = int(1000 * (y0 / height))
  y2 = int(1000 * (y2 / height))

  return [x0, y0, x2, y2]


def write_dataset(dataset: list, output_dir: Path, name: str):
  print(f"Writing {name}ing dataset:")
  with open(output_dir / f"{name}.txt", "w+", encoding="utf8") as file, \
       open(output_dir / f"{name}_box.txt", "w+", encoding="utf8") as file_bbox, \
       open(output_dir / f"{name}_image.txt", "w+", encoding="utf8") as file_image:

      # Go through each dataset
      for datas in tqdm(dataset, total=len(dataset)):
        data, width, height = datas
        
        filename = data.iloc[0, data.columns.get_loc('filename')]

        # Go through every row in dataset
        for index, row in data.iterrows():
          bbox = [int(p) for p in row[['x0', 'y0', 'x2', 'y2']]]
          normalized_bbox = normalize(bbox, width, height)

          file.write("{}\t{}\n".format(row['line'], row['label']))
          file_bbox.write("{}\t{} {} {} {}\n".format(row['line'], *normalized_bbox))
          file_image.write("{}\t{} {} {} {}\t{} {}\t{}\n".format(row['line'], *bbox, width, height, filename))

        # Write a second newline to separate dataset from others
        file.write("\n")
        file_bbox.write("\n")
        file_image.write("\n")

In [14]:
dataset_directory = Path('working','dataset')

dataset_directory.mkdir(parents=True, exist_ok=True)

write_dataset(dataset_train, dataset_directory, 'train')
write_dataset(dataset_test, dataset_directory, 'test')

# Creating the 'labels.txt' file to the the model what categories to predict.
labels = ['COMPANY', 'DATE', 'ADDRESS', 'TOTAL']
IOB_tags = ['S']
with open(dataset_directory / 'labels.txt', 'w') as f:
  for tag in IOB_tags:
    for label in labels:
      f.write(f"{tag}-{label}\n")
  # Writes in the last label O - meant for all non labeled words
  f.write("O")

Writing training dataset:


100%|██████████| 626/626 [00:08<00:00, 71.67it/s]


Writing testing dataset:


100%|██████████| 347/347 [00:04<00:00, 72.30it/s]


In [ ]:
# %%bash
# git clone https://github.com/microsoft/unilm.git
# cd unilm/layoutlm/deprecated
# pip install .

bash: line 2: cd: unilm/layoutlm/deprecated/setup.py: No such file or directory
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


CalledProcessError: Command 'b'#git clone https://github.com/microsoft/unilm.git\ncd unilm/layoutlm/deprecated/setup.py\npip install .\n'' returned non-zero exit status 1.

In [ ]:
pretrained_model_folder_input= Path('SROIE2019') # Define it so we can copy it into our working directory

pretrained_model_folder=Path('working/unilm/layoutlm-base-uncased/') 
label_file=Path(dataset_directory, "labels.txt")

# Move to the script directory
os.chdir("wroking/unilm/layoutlm/deprecated/examples/seq_labeling")

In [ ]:
! cp -r "{pretrained_model_folder_input}" "{pretrained_model_folder}"
! sed -i 's/"num_attention_heads": 16,/"num_attention_heads": 12,/' "{pretrained_model_folder}/"config.json

cp: SROIE2019: No such file or directory
sed: 1: "unilm/layoutlm-base-unc ...": invalid command code u


In [ ]:
! cat "working/unilm/layoutlm-base-uncased/config.json"

cat: unilm/layoutlm-base-uncased/config.json: No such file or directory


In [ ]:
! rm -rf /kaggle/working/dataset/cached*

In [ ]:
! python run_seq_labeling.py \
                            --data_dir working/dataset \
                            --labels working/dataset/labels.txt \
                            --model_name_or_path "{pretrained_model_folder}" \
                            --model_type layoutlm \
                            --max_seq_length 512 \
                            --do_lower_case \
                            --do_train \
                            --num_train_epochs 10 \
                            --logging_steps 50 \
                            --save_steps -1 \
                            --output_dir output \
                            --overwrite_output_dir \
                            --per_gpu_train_batch_size 8 \
                            --per_gpu_eval_batch_size 16

In [ ]:
# Evaluate for test set and make predictions
! python run_seq_labeling.py \
                            --data_dir working/dataset \
                            --labels working/dataset/labels.txt \
                            --model_name_or_path "{pretrained_model_folder}" \
                            --model_type layoutlm \
                            --do_lower_case \
                            --max_seq_length 512 \
                            --do_predict \
                            --logging_steps 10 \
                            --save_steps -1 \
                            --output_dir output \
                            --per_gpu_eval_batch_size 8

In [ ]:
!pip freeze

In [ ]:
cat output/test_results.txt

In [ ]:
import cv2
from matplotlib import pyplot, patches
import matplotlib

data = pd.read_csv("working/dataset/test_image.txt", delimiter="\t", names=["name", "bbox", "size", "image"])
data_category = pd.read_csv("working/dataset/test.txt", delimiter="\t", names=["name", "true_category"]).drop(columns=["name"])
data_prediction_category = pd.read_csv("output/test_predictions.txt", delimiter=" ", names=["name", "prediction_category"]).drop(columns=["name"])

data_merge = data.merge(data_category, left_index=True, right_index=True)
merged = data_merge.merge(data_prediction_category, left_index=True, right_index=True)
merged_groups = list(merged.groupby("image"))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/dataset/test_image.txt'

In [ ]:
def display_prediction(data, file):
  colors = {
      "S-TOTAL": (255,0,0),
      "S-DATE": (0,255,0),
      "S-ADDRESS": (0,0, 255),
      "S-COMPANY": (255,255,0),
      "O": (192,192,192)
  }


  imagename = data[0].split(".")[0] + ".jpg"
  print("Filename:",imagename)
  image_path = str('SROIE2019/test/img/X51005200931.jpg')

  img=cv2.imread(image_path)
  img_prediction=cv2.imread(image_path)

  data = data[1]
  for bbox, category, prediction_category in zip(data['bbox'], data['true_category'], data['prediction_category']):
    (x1, y1, x2, y2) = [int(coordinate) for coordinate in bbox.split()]

    img_prediction = cv2.rectangle(img_prediction, (x1, y1), (x2, y2), colors[prediction_category], 2 if "O" in prediction_category else 4)
    img = cv2.rectangle(img, (x1, y1), (x2, y2), colors[category], 2 if "O" in category else 4)

  matplotlib.rcParams['figure.figsize'] = 15 ,18

  cv2.imwrite("prediction.jpg", img_prediction)

  # Plot
  fig, ax = matplotlib.pyplot.subplots(1,2)
  ax[0].set_title("Original", fontsize= 30)
  ax[0].imshow(img);
  ax[1].set_title("Prediction", fontsize= 30)
  ax[1].imshow(img_prediction);

  # Legend
  handles = [
      patches.Patch(color='yellow', label='Company'),
      patches.Patch(color='blue', label='Address'),
      patches.Patch(color='green', label='Date'),
      patches.Patch(color='red', label='Total'),
      patches.Patch(color='gray', label='Other')
  ]

  fig.legend(handles=handles, prop={'size': 25}, loc='lower center')

In [ ]:
display_prediction(merged_groups[0], 'test')

In [ ]:
display_prediction(merged_groups[34], 'test')